In [28]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [29]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [30]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_llama = env_to_str('MODEL_LLAMA3')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [31]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [32]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="ollama_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

Overriding of current TracerProvider is not allowed
DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ollama_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [33]:
# Ollama Model
ollama_model = OpenAIChatCompletionsModel(
    model=model_llama,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [34]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=ollama_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=ollama_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=ollama_model
)

In [35]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agents):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name):
        if agents:
            for index, agent in enumerate(agents):
                # print(f"  Argument {index+1}: {agent}")
        
                # # Trace Attributes
                # current_span = otel_trace.get_current_span()
                # current_span.set_attribute("user.request", message)
                # current_span.set_attribute("agent.name", agent.name)
                # current_span.set_attribute("model.name", model_llama)
                results = await asyncio.gather(
                    *[Runner.run(agent, message) for agent in agents]
                )

        else:
            print("No additional arguments provided.")

        outputs = [result.final_output for result in results]
        # result = await Runner.run(agents, message)
        
        # # Add response attributes
        # current_span.set_attribute("response.length", len(result.final_output))
        # current_span.set_attribute("response.preview", result.final_output[:100])
        for output in outputs:
            display(Markdown(output))
        return outputs

In [36]:
result = await phoenix_tracer(
    "Parallel Cold Emails", 
    "Write a cold sales email.", 
    sales_agent1, 
    sales_agent2, 
    sales_agent3
)

Here's a sample cold sales email:

Subject: Ensure Confidence in Your Organization's Compliance Process

Dear [Recipient],

As a [Company Title] at [Company Name], I'm reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to simplify and streamline your organization's SOC2 compliance journey.

In today's complex regulatory landscape, ensuring the security and integrity of sensitive client data is not just a best practice – it's a business necessity. As an [Industry/Market] provider, I understand that maintaining rigorous audit protocols is critical to upholding the trust of clients and partners alike.

ComplAI's innovative platform leverages artificial intelligence (AI) to guide your organization through every step of the SOC2 compliance process, from scoping to reporting. With our tool, you'll gain unparalleled visibility into assessment activities, instantaneously generate comprehensive reports, and enjoy effortless audit prep capabilities – all in a secure, cloud-based environment.

Our valued clients achieve significant benefits, including:

* 90% reduction in manual effort
* 50% decrease in project timelines
* Enhanced transparency and stakeholder confidence

If you're struggling to manage SOC2 compliance or experiencing difficulties with audit preparation, I'd like to schedule a brief consultation to discuss how ComplAI can help. Your organization deserves assurance that its compliance processes are secure, efficient – and built on the latest technology advancements.

Would you be available for a 15-minute call to explore your current challenges and review our tailored approach? Please let me know if you're interested or prefer additional information at your convenience.

Looking forward to helping you unlock confidence in your organization's compliance process with ComplAI.

Best regards,

[Your Name]
Sales Agent, ComplAI
[Contact Information]

Here's a cold sales email:

**Subject: Alert: Your Security Audit is Coming! Are You Ready? **

Dear [First Name],

 Congrats! Your company is growing rapidly, which means you're probably about to earn the elusive SOC2 designation - a badge of honor that says "We take security seriously!" 

But before the audit crew arrives with their clipboards and questioning faces... Have you reviewed your compliance documentation lately? We didn't think so!

Introducing ComplAI: Your AI-powered SOC2 compliance sidekick! Our SaaS tool helps ensure you're audit-ready, saving you hours of paperwork drudgery (yes, that's a thing).

With ComplAI, you'll get:

 Customized compliance checklists
 Automated document review and updating
 Real-time monitoring of your security controls

Don't let one incorrect checkbox or outdated report ruin the party! Schedule a demo with me to see how ComplAI can save you from audit anxiety and position your company for long-term success.

Hit reply, give me a quick call, or schedule a time that works best for you here: [Calendar link]

Hope to chat soon!

Best regards,

[Your Name]
ComplAI Team

P.S. Did we mention we have a 30-day money-back guarantee? No strings attached. Just audit readiness and peace of mind!

This email aims to:

* Grab attention with a lighthearted and relevant hook (SOC2 audit coming soon!)
* Introduce ComplAI as the solution to prevent audit anxiety
* Highlight key features and benefits in 3 bullet points
* Create urgency and encouragement for scheduling a demo
* End on a friendly note with a P.S. section

Here's a cold sales email:

Subject: Ensure Seamless SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

As a service provider, you understand the importance of maintaining strict compliance standards to protect sensitive client data and maintain trust in your business.

Are you tired of tedious audit preparations? Are security frameworks like NIST CSP and PCI-DSS becoming overwhelming?

Our AI-powered SaaS tool at ComplAI simplifies SOC 2 compliance. In under 30 minutes, you can:

* Generate comprehensive audit reports
* Automate documentation and evidence collection
* Receive real-time data on your compliance status

Want to experience the ease of compliant operations? Schedule a demo with us and discover how ComplAI can elevate your SOC 2 game.

Best regards,

[Your Name]
Sales Agent, ComplAI